In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
booked_calls = pd.read_csv('data/Booked Calls.csv')
call_outcomes = pd.read_csv('data/Call Outcomes.csv')




call_outcomes.rename(columns={'Email Address': 'Email '}, inplace=True)


print(booked_calls.columns)
print(call_outcomes.columns)




Index(['Schedule Date', 'Appoinment Date', 'First Name', 'Last Name ',
       'Email ', 'Phone ', 'Closer', 'Source', 'Source - RR / FC',
       'Qualified?', 'Outcome Documented?', 'Showed?', 'Close / No Close',
       'Call Outcome ', 'Lost Reason', 'Deposit Amount', 'Payment Plan',
       'Cash Collected ', 'Revenue', 'Product', 'Call Recording Link',
       'Notes'],
      dtype='object')
Index(['Date', 'First Name', 'Last Name', 'Email ', 'Closer', 'Call Outcome ',
       'Showed?', 'Lost Reason', 'Deposit Amount', 'Payment Plan',
       'Cash Collected ', 'Revenue', 'Product', 'Call Recording Link',
       'Notes'],
      dtype='object')


In [13]:
# last 3 months

# Parse 'Schedule Date' robustly (handles mixed formats like 2/22/2025 and 28/7/2025)
schedule_dates = pd.to_datetime(booked_calls["Schedule Date"], errors="coerce", dayfirst=False)
needs_dayfirst = schedule_dates.isna()
if needs_dayfirst.any():
    schedule_dates.loc[needs_dayfirst] = pd.to_datetime(
        booked_calls.loc[needs_dayfirst, "Schedule Date"], errors="coerce", dayfirst=True
    )
booked_calls["Schedule Date"] = schedule_dates

# Compute threshold for last 3 months
three_months_ago = pd.Timestamp.today().normalize() - pd.DateOffset(months=3)

# Apply date filter correctly
recent_booked = booked_calls[booked_calls["Schedule Date"] >= three_months_ago]

# Filter by source containing 'youtube'
matcher = recent_booked["Source"].str.contains("youtube", case=False, na=False)
youtube_leads = recent_booked.loc[matcher].copy()

# Merge outcomes
merged = pd.merge(youtube_leads, call_outcomes, on="Email ", how="left", suffixes=("_x", "_y"))

# Drop unneeded columns if present
cols_to_drop = [
    "Appoinment Date",
    "Schedule Date",
    "First Name_y",
    "Last Name _y",
    "Product_x",
]
merged.drop(columns=[c for c in cols_to_drop if c in merged.columns], inplace=True)

# Exclude specific sources
merged = merged[~merged["Source"].isin([
    "ghl | live_by_design | referral | redirect_from_homepage",
    "ghl | live_by_design | referral | redirect_from_old_vsl",
])]

# Won deals
won = merged[merged["Call Outcome _y"].str.contains("Won", case=False, na=False)]
print(youtube_leads.columns)
youtube_leads.drop(columns=["Email ","Phone "], inplace=True)
youtube_leads.to_csv("output/youtube_leads.csv", index=False)
print(youtube_leads)









Index(['Schedule Date', 'Appoinment Date', 'First Name', 'Last Name ',
       'Email ', 'Phone ', 'Closer', 'Source', 'Source - RR / FC',
       'Qualified?', 'Outcome Documented?', 'Showed?', 'Close / No Close',
       'Call Outcome ', 'Lost Reason', 'Deposit Amount', 'Payment Plan',
       'Cash Collected ', 'Revenue', 'Product', 'Call Recording Link',
       'Notes'],
      dtype='object')
    Schedule Date Appoinment Date   First Name   Last Name           Closer  \
319    2025-06-05     10 May 2025       Harris       Saleem  Muhammad Axmed   
320    2025-06-05      8 May 2025       Khaled     Mohammed  Muhammad Axmed   
321    2025-06-05     10 May 2025     Ishtiyaq      Ebrahim  Muhammad Axmed   
322    2025-07-05     10 May 2025      Mohamed        Amine  Muhammad Axmed   
323    2025-07-05     11 May 2025       Abdoul            K  Muhammad Axmed   
..            ...             ...          ...          ...             ...   
458    2025-07-24    27 July 2025        Binah  Abe